## GOAL 
the goal of this notebook is to create a metadata file that has annotation about genes in the R. pal genome
We scrape this from a variety of sources. It will be put into a dataframe (Pandas) and that can be
merged later with omics data if you want to do some further research


## Notebook Outline

1. import statements
2. ID mapping via a genbank parse
4. COG - get a functional annotatino and categorization for each protein
5. Gene essentiality


In [1]:
#### Step 1 - imports ####

from pandas import DataFrame, read_table
import pandas as pd
from Bio import SeqIO   # used to parse the genbank file

In [2]:
#### Step 2 - Identifier mapping #####
## Our goal here is to Parse the R pal genbank files 
## to create a mapping between RefSeq accessions and the locus tag
## specifically the old_locus_tag because that is what is in the paper https://www.ncbi.nlm.nih.gov/pubmed/26712940


Rpal_genbank_file = "R_pal_aux_files\R_pal_chromosome_NC_005296.gb"
Rpal_chromosome = SeqIO.read(Rpal_genbank_file, "genbank")
Rpal_data = [] #temp storage for later merging

for feature in Rpal_chromosome.features:
    #step 1. I only care about CDS features and not gene features (which lack a lot of information)
    if not feature.type == 'CDS':
        continue
    #print (feature)
    # step 2, because I am working with the Essentail genes paper, I only really care
    # about proteins that I can map back to that paper using the old_locus_tag
    if not 'old_locus_tag' in feature.qualifiers:
        continue
    if not 'protein_id' in feature.qualifiers:
        continue
    ### Okay, now to things we care about
    acc = feature.qualifiers['protein_id'][0]# get the zero-th element in a list of len 1
    locus = feature.qualifiers['old_locus_tag'][0]

    gene_product = feature.qualifiers['product'][0]
    ### As an added bonus, we can probably get a COG out of most of these
    COG = None #default value
    if 'function' in feature.qualifiers:
        functions = feature.qualifiers["function"][0] #list len 1. again.
        #now parse out the function
        ##   InterPro IPR000485 COGs COG1522
        ##   COGs COG0526
        ##################### WARNING. hack below ##################
        # some parsing based on what I've seen before. not robust parsing to other formats
        #print (functions)
        Bits = functions.split(" ") #split on space
        if Bits[0] == "COGs":
            COG = Bits[1]
        if len(Bits)>2:
            if Bits[2] == "COGs":
                COG = Bits[3]
    ### Now that I've done some parsing, we should gather this
    Rpal_data.append({'RefSeq':acc, 'COG':COG, 'locus':locus, 'gene_product':gene_product}) # I am creating a list of dictionaries. 
    
    
#now after the file parsing loop, make my dataframe 'df'
df = pd.DataFrame(data=Rpal_data)


In [3]:
#### Step 3 - COG  ####
#### Step 3.1 COG identifiers matched to COG categories
## the goal here is to parse the COG type file
COG_categories_file = "COG\cognames2003-2014.txt"
Handle = open(COG_categories_file, 'r')
Header = Handle.readline()

COG_data = [] #list of dictionaries

for line in Handle:
    Bits = line.strip().split("\t")
    COG = Bits[0]
    category = Bits[1]
    if len(category)> 1:
        #print (category)
        category = category[0] # hack for the minute, because I don't want to have something with multiple categories
    COG_data.append({'COG':COG, 'COG_category':category})
Handle.close()

#now make a temp data frame and merge it back into the big one.
df_temp = pd.DataFrame(data=COG_data)
df = df.merge(df_temp, left_on='COG', right_on="COG", how='left')


In [4]:
#### step 3.2 - COG meta categories ####

##we want a simpler column for COG category, something like a meta-category
meta_category = []
#metabolite meta category  'M'
things_in_M = ['G', 'E', 'F', 'H', 'I', 'P', 'Q', 'C']
things_in_M.sort()
for thing in things_in_M:
    meta_category.append({'COG_category':thing, 'COG_meta':'M'})
#transcription translation meta category 'T'
things_in_T = ['J', 'A', 'K', 'L', 'B', 'D']
things_in_T.sort()
for thing in things_in_T:
    meta_category.append({'COG_category':thing, 'COG_meta':'T'})
#other meta category 'O'
things_in_O = ['V', 'T', 'M', 'N', 'U', 'O', 'R', 'S']
things_in_O.sort()
for thing in things_in_O:
    meta_category.append({'COG_category':thing, 'COG_meta':'O'})
    
##Categories that didn't have anything, so they are left out
# X Mobilome: prophages, transposons
# Z Cytoskeleton
# W Extracellular structures
# Y Nuclear structure
    
#meta_category.append({'COG_category':'EH', 'COG_meta':'M'})#there are so many two letter combos. Not sure how to deal with it.

#now merge this new meta_category stuff into the main data frame df
df_temp = pd.DataFrame(data=meta_category)
df = df.merge(df_temp, left_on='COG_category', right_on="COG_category", how='left')


In [5]:
#### Step 4 - essential genes ####
#### Step 4.1 - essential genes from PMID 26712940 - a paper about aerobic growth

Rpal_essentials_file = "R_pal_aux_files\R_pal_essential_PMID_26712940.txt"
Handle = open(Rpal_essentials_file, 'r')
Header1 = Handle.readline()
Header2 = Handle.readline() # it has two header lines. whatever.
Essential_data = [] #temp storage for later merging
#all we want is the list of whether something is essential or not
#everything in this file *is essential*, so that's why it's hardcoded
for line in Handle.readlines():
    Bits = line.strip().split("\t")
    Rpal_locus_id = Bits[0]
    Essential_data.append({'Essential_aerobic':True ,'locus':Rpal_locus_id})
    
Handle.close()
df_temp = pd.DataFrame(data=Essential_data) #now make a temp data frame and merge it back into the big one.
df = df.merge(df_temp, left_on='locus', right_on="locus", how='left')
## the default value of merging is to put 'NaN' for null. This is a problem for plotting and logic stuff
df['Essential_aerobic'].fillna(False, inplace=True)


In [6]:
#### Step 4.2 - essential genes from PMID 28677146 - a paper about phototrophic growth
## this list is pre-filtered by Carrie Harwood to have no overlap with the aerobic growth

Rpal_essentials_file = "R_pal_aux_files\R_pal_essential_PMID_28677146.txt"
Handle = open(Rpal_essentials_file, 'r')
Header1 = Handle.readline()
Header2 = Handle.readline() # it has two header lines. whatever.
Essential_data = [] #temp storage for later merging
#all we want is the list of whether something is essential or not
#everything in this file *is essential*, so that's why it's hardcoded
for line in Handle.readlines():
    Bits = line.strip().split("\t")
    Rpal_locus_id = Bits[0]
    Essential_data.append({'Essential_phototrophic':True ,'locus':Rpal_locus_id})
    
Handle.close()
df_temp = pd.DataFrame(data=Essential_data) #now make a temp data frame and merge it back into the big one.
df = df.merge(df_temp, left_on='locus', right_on="locus", how='left')
## the default value of merging is to put 'NaN' for null. This is a problem for plotting and logic stuff
df['Essential_phototrophic'].fillna(False, inplace=True)


In [7]:
#### Step 4.3 - essential Genes for PMID 29184015  - a paper about longevity
## this list is pre-filtered by Carrie Harwood to have no overlap with the aerobic growth

Rpal_essentials_file = "R_pal_aux_files\R_pal_essential_PMID_29184015.txt"
Handle = open(Rpal_essentials_file, 'r')
Header1 = Handle.readline()
Header2 = Handle.readline() # it has two header lines. whatever.
Essential_data = [] #temp storage for later merging

for line in Handle.readlines():
    Bits = line.strip().split("\t")
    #for starters, all we want is the list of whether something is essential or not
    Rpal_locus_id = Bits[0]
    #everything in this file *is essential*, so that's why it's hardcoded
    Essential_data.append({'Essential_longevity':True ,'locus':Rpal_locus_id})
    
Handle.close()
#now make a temp data frame and merge it back into the big one.
df_temp = pd.DataFrame(data=Essential_data)
df = df.merge(df_temp, left_on='locus', right_on="locus", how='left')
## the default value of merging is to put 'NaN' for null. This is a problem if you want to explicity
## use the True and False keywords for a binary column. The 'NaN' will not get plotted.
df['Essential_longevity'].fillna(False, inplace=True) # have to fill in the value, or else it will not plot the 'false' group


In [8]:
##### Step 5 - Pfam

#now trying to merge in the PFam assignments which are made via Uniprot,
#which has to get merged from the locus name
Pfam_array = []
from Bio import SwissProt
handle = open('R_pal_aux_files\R_pal_proteome.sp')
for record in SwissProt.parse(handle):
    gene_name = record.gene_name
    locus = None
    #Name=hbaA; OrderedLocusNames=RPA0669;
    for bit in gene_name.split(";"):
        if bit[:7] == 'Ordered':
            (blah, locus) = bit.split("=")
            # occasionally we get something like OrderedLocusNames=RPA3337 {ECO:0000313|EMBL:CAE28778.1};
            if ' ' in locus:
                locus = locus.split(' ')[0] #split on the space and take the first thing
            break
    if not locus:
        #we don't have anything to merge on later, so we might as well quit
        continue
    DRs = record.cross_references # a list of tuples
    for DR in DRs:
        if DR[0] == 'Pfam': # now we care
            (Pfam_ID, Pfam_name) = (DR[1], DR[2])
            Pfam_array.append({'locus':locus, 'Pfam_ID':Pfam_ID, 'Pfam_name':Pfam_name})
df_temp = pd.DataFrame(data=Pfam_array)

df = df.merge(df_temp, left_on='locus', right_on='locus', how='left')


In [9]:
#### save dataframe ####
out_filename = "R_pal_metadata_df.txt"
df.to_csv(out_filename, sep="\t", index=False)

